In [2]:
import pandas as pd

In [18]:
import requests

## df_ids 

df_ids created to get the ids of the spotify top songs 

In [3]:
df = pd.read_csv("../data/df_spotify.csv", index_col = 0 )

In [6]:
"https://open.spotify.com/embed?uri=spotify:track:"+df["Song_Id"][0]

'https://open.spotify.com/embed?uri=spotify:track:3Wrjm47oTz2sjIgck11l5e'

In [7]:
pd.options.display.max_columns = None

In [14]:
df1=df[["Song_Name", "Song_Id"]]

In [15]:
df1.to_csv("../data/df_id.csv")

## df_ticketmaster_artist

Using ticketmaster data I will enrich the information from events by artist with its location and price

In [90]:
df = pd.read_csv("../data/df_visualisacion.csv", index_col = 0 )

In [3]:
df.head()

,Song_Name,Genre,Subgenre,Subgenre_2,Release_Date,Song_Decade,Streams,Week_Of_Highest_Charting,Month_Highest_Charting,Season_Highest_Charting,...,Gender,Artist_Generation,Type,Country,Event_Name,url,Event_Country,Event_City,Event_Longitude,Event_Latitude
0,UGH!,pop,k-pop,k-pop boy group,2020-02-21,2020s,"9,991,177",2020-02-21,February,Winter,...,mixed,Generation Z,group,KR,BTS 14-17 Party,https://www.ticketmaster.co.uk/bts-1417-party-...,Great Britain,London,-0.10661535,51.53445948
1,Bigger Than Life,rap,melodic rap,philly rap,2020-03-06,2020s,"9,974,143",2020-03-06,March,Spring,...,male,Millennial,person,US,Rolling Loud California,https://on.fgtix.com/trk/Ntkn,United States Of America,Inglewood,-118.3368854,33.95371247
2,Time Flies,hip hop,canadian hip hop,canadian pop,2020-05-01,2020s,"9,904,749",2020-05-01,May,Spring,...,male,Millennial,person,CA,Drake White,https://www.ticketmaster.com/drake-white-india...,United States Of America,Indianapolis,-86.249381,39.768343
3,HOT WIND BLOWS (feat. Lil Wayne),hip hop,hip hop,rap,2021-06-25,2020s,"9,889,197",2021-06-25,June,Summer,...,male,Millennial,person,US,"EARFQUAKE! A Tyler, The Creator Party with Bla...",https://www.ticketweb.com/event/earfquake-a-ty...,United States Of America,Fresno,-119.8023913,36.7589448
4,Wonderful Christmastime - Edited Version / Rem...,rock,album rock,art rock,1980-05-16,1980s,"9,882,989",2020-12-18,December,Winter,...,male,Baby Boomer,person,GB,Tony Kishman-Live and Let Die: The Music of Pa...,https://www.ticketmaster.com/tony-kishmanlive-...,United States Of America,Fort McDowell,-111.67075,33.67495


In [129]:
def request_ticketmaster_spoti2(artistas, my_token): 
    events=[]
    token= my_token
    url = "https://app.ticketmaster.com/discovery/v2/events/"
    for artist in artistas:
        params = {"keyword": artist,
               "apikey":token}   
        res = requests.get(url,params=params)
        if res.status_code == 200:
            data = res.json()
        else:
            print(f"Tenemos un problema en el video event")
        artist = artist
        try:
            event1=data['_embedded']['events'][0]
            event_name = event1['name']
            event_url = event1['url']
            event_city= event1["_embedded"]['venues'][0]['city']["name"]
            event_country= event1["_embedded"]['venues'][0]['country']["name"]
            event_location= event1["_embedded"]['venues'][0]['location']
            event_date=event1['dates']["start"]["localDate"]
            try:
                event_minprice=event1['priceRanges'][0]["min"]
                event_maxprice=event1['priceRanges'][0]["max"]
                event_div=event1['priceRanges'][0]["currency"]
            except:
                event_minprice="Unknown"
                event_maxprice="Unknown"
                event_div="Unknown"
                
            events.append({'Artist': artist, 'Event_Name': event_name, 'url': event_url, "Country": event_country, "City":event_city, "Date": event_date, "Max_price":event_maxprice, "Min_price":event_minprice, "Currency":event_div, "Location": event_location})
        except:
            events.append({'Artist': artist, 'Event_Name': "No events", 'url': "-", "Country": "-", "City":"-", "Location": "-", "Date": "-", "Max_price":"-", "Min_price":"-", "Currency":"-", "Location": "-"})

    return events

In [115]:
Artist=df["Artist"].unique()

In [130]:
df_api=request_ticketmaster_spoti2(Artist, token)

In [131]:
df_api2=pd.DataFrame(df_api)

In [133]:
df_api2

,Artist,Event_Name,url,Country,City,Date,Max_price,Min_price,Currency,Location
0,BTS,BTS Club Night,https://www.ticketweb.uk/event/bts-club-night-...,Great Britain,London,2023-04-14,Unknown,Unknown,Unknown,"{'longitude': '-0.103855', 'latitude': '51.546..."
1,Lil Uzi Vert,Longitude 2023 - Sunday Ticket - Travis Scott ...,https://www.ticketmaster.ie/longitude-2023-sun...,Ireland,Dublin,2023-07-02,107.05,107.05,EUR,"{'longitude': '-6.26819372', 'latitude': '53.2..."
2,Drake,Drake,https://www.ticketmaster.com/event/Z7r9jZ1Adqfea,United States Of America,Glendale,2023-07-06,Unknown,Unknown,Unknown,"{'longitude': '-112.2202', 'latitude': '33.5312'}"
3,"Tyler, The Creator",No events,-,-,-,-,-,-,-,-
4,Paul McCartney,Tony Kishman-Live and Let Die: The Music of Pa...,https://www.ticketmaster.com/tony-kishmanlive-...,United States Of America,Fort McDowell,2023-02-18,75.0,35.0,USD,"{'longitude': '-111.67075', 'latitude': '33.67..."
...,...,...,...,...,...,...,...,...,...,...
315,Tiësto,Tiesto,https://www.ticketmaster.com/event/Z7r9jZ1AdQ87o,United States Of America,Las Vegas,2023-02-11,Unknown,Unknown,Unknown,"{'longitude': '-115.145103', 'latitude': '36.1..."
316,Daryl Hall & John Oates,No events,-,-,-,-,-,-,-,-
317,The Ronettes,No events,-,-,-,-,-,-,-,-
318,Band Aid,No events,-,-,-,-,-,-,-,-


In [134]:
df_api3=df_api2[df_api2["Event_Name"]!="No events"]

In [135]:
df_api3.to_csv("../data/df_ticketmaster_artist.csv")